<a href="https://colab.research.google.com/github/drpetros11111/transformers-with-python/blob/main/02_sentiment_with_transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentiment with Transformers

The HuggingFace Transformers library is presently the most advanced and accessible library for building and using transformer models. As such, it will be what we primarily use throughout these notebooks.

To apply sentiment analysis using the transformers library, we first need to decide on a model to use - as we will be applying a pretrained model, rather than starting from scratch. The list of models available can be found at [huggingface.co/models](https://www.huggingface.co/models).

![Filter for Text Classification on HuggingFace models page](assets/hf_models_text_classification_filter.jpg)

From the model page we select the **Text Classification** filter on the left of the page to filter for models that we can apply for text classification tasks immediately. We will be performing sentiment analysis on posts from */r/investing* (in this section we will be using the example given in `txt` below), which are finance oriented, so we can use the [finBERT](https://arxiv.org/abs/1908.10063) model [`ProsusAI/finbert`](https://huggingface.co/ProsusAI/finbert) which has been trained on financial articles for financial sentiment classification.

FinBERT is ofcourse a BERT model, so when loading the model and tokenizer we will using BERT classes, and because we are performing *sequence classification* we will be using `BertForSequenceClassification`. Let's initialize our model and tokenizer:

In [ ]:
from transformers import BertForSequenceClassification, BertTokenizer

# initialize the tokenizer for BERT models
tokenizer = BertTokenizer.from_pretrained('ProsusAI/finbert')
# initialize the model for sequence classification
model = BertForSequenceClassification.from_pretrained('ProsusAI/finbert')

# Sentiment Analysis with BERT
This code is about importing and initializing a pre-trained BERT model and tokenizer for sentiment analysis, specifically using the FinBERT model which is tailored for financial text.

---------------------------

#Import necessary modules:

    from transformers import BertForSequenceClassification, BertTokenizer

##`transformers`:
This is the Hugging Face Transformers library, offering tools to work with pre-trained models.

##`BertForSequenceClassification`:

This specific class is used when you want to use BERT for tasks like sentiment classification, where you need to assign a label or category to a piece of text.

##`BertTokenizer`:
This class is used to convert raw text into the numerical input format that BERT understands.

-------------------
This process is called tokenization.

#Initialize the Tokenizer:

    tokenizer = BertTokenizer.from_pretrained('ProsusAI/finbert')

##`tokenizer`:
 A variable to store the tokenizer object.

##`from_pretrained('ProsusAI/finbert')`:
This loads a pre-trained tokenizer specific to the FinBERT model.

This tokenizer is trained on financial text and knows how to break it down into tokens.

This will download the necessary files if you don't have them already.

----------------------
#Initialize the Model:

    model = BertForSequenceClassification.from_pretrained('ProsusAI/finbert')


##`model`:
A variable to store the BERT model object.

##`from_pretrained('ProsusAI/finbert')`:
 This line loads the pre-trained FinBERT model.

 FinBERT is a BERT model that's fine-tuned on financial data, making it better suited for financial sentiment analysis.

 Like the tokenizer, this will download the model weights if they are not already present.

In essence, this code does the following:

--------------------
#Gets the tools (classes):

Imports BertForSequenceClassification to do the sentiment classification and BertTokenizer to prepare the text.

##Sets up the tokenizer:

Creates a tokenizer object using the FinBERT tokenizer to break down text.

##Sets up the model:

Creates a model object using the pre-trained FinBERT model, ready for sentiment analysis on financial text.

The first time that this is run when using the `ProsusAI/finbert` model, it will be downloaded from the HuggingFace model repositories. We will be following a very similar process to that which we worked through for our Flair sentiment classifier, with some added steps to convert model output activations to class predictions.

1. We tokenize our input text.

2. Tokenized inputs are fed into the model, which outputs final layer **activations** (note *activations* are not *probabilities*).

3. Convert those activations into probabilities using a softmax function (sigmoid for multiple classes).

4. Take the **argmax** of those probabilities.

5. *(Optional) Extract the probability of the winning class.*

For step one, we will use the `encode_plus` method:

In [ ]:
# this is our example text
txt = ("Given the recent downturn in stocks especially in tech which is likely to persist as yields keep going up, "
       "I thought it would be prudent to share the risks of investing in ARK ETFs, written up very nicely by "
       "[The Bear Cave](https://thebearcave.substack.com/p/special-edition-will-ark-invest-blow). The risks comes "
       "primarily from ARK's illiquid and very large holdings in small cap companies. ARK is forced to sell its "
       "holdings whenever its liquid ETF gets hit with outflows as is especially the case in market downturns. "
       "This could force very painful liquidations at unfavorable prices and the ensuing crash goes into a "
       "positive feedback loop leading into a death spiral enticing even more outflows and predatory shorts.")

tokens = tokenizer.encode_plus(txt, max_length=512, truncation=True, padding='max_length',
                               add_special_tokens=True, return_tensors='pt')

entokens

{'input_ids': tensor([[  101,  2445,  1996,  3522,  2091, 22299,  1999, 15768,  2926,  1999,
          6627,  2029,  2003,  3497,  2000, 29486,  2004, 16189,  2562,  2183,
          2039,  1010,  1045,  2245,  2009,  2052,  2022, 10975, 12672,  3372,
          2000,  3745,  1996, 10831,  1997, 19920,  1999, 15745,  3802, 10343,
          1010,  2517,  2039,  2200, 19957,  2011,  1031,  1996,  4562,  5430,
          1033,  1006, 16770,  1024,  1013,  1013,  1996,  4783,  2906, 27454,
          1012,  4942,  9153,  3600,  1012,  4012,  1013,  1052,  1013,  2569,
          1011,  3179,  1011,  2097,  1011, 15745,  1011, 15697,  1011,  6271,
          1007,  1012,  1996, 10831,  3310,  3952,  2013, 15745,  1005,  1055,
          5665, 18515, 21272,  1998,  2200,  2312,  9583,  1999,  2235,  6178,
          3316,  1012, 15745,  2003,  3140,  2000,  5271,  2049,  9583,  7188,
          2049,  6381,  3802,  2546,  4152,  2718,  2007,  2041, 12314,  2015,
          2004,  2003,  2926,  1996,  

# Explaing the Tokens

---------------------------------
##Token IDs:
BERT uses a vocabulary where each word or subword is assigned a unique numerical ID.

These IDs are what the numbers in the tensor represent.

----------------------------------------
##Special Tokens:

Certain token IDs have specific meanings:

##101:
Represents the start of a sequence ([CLS] token)

##102:
Represents the end of a sequence or separation between sequences ([SEP] token)

##0:
Represents padding tokens ([PAD] tokens)

##Padding:
BERT models often require input sequences to be of a fixed length.

If the original sequence is shorter, padding tokens (represented by 0s) are added to bring it to the required length.

------------------------------------------
#How it relates to your tensor:

The tensor starts with 101, indicating the start of the sequence.

It contains a series of numbers, each representing a word or subword in your input text.

These numbers correspond to the token IDs in the BERT vocabulary.

##You can see 102, which signifies the end of the actual sequence.

The rest of the tensor is filled with 0s.

This means the original text sequence was shorter than the required input length, so padding tokens were added to fill the gap.

------------------------------------
#Attention Mask
The attention_mask tells the attention layers in BERT which words to calculate attention for.

Here's a breakdown:

##Connection to input_ids:

Each 1 value in the attention_mask directly corresponds to an actual token (a word or subword) in the input_ids tensor.

##Ignoring Padding:

Conversely, each 0 in the attention_mask corresponds to a padding token (0) in the input_ids tensor.

##Attention Calculation:

During the attention mechanism's calculations, the activations (values) associated with padding tokens are essentially multiplied by 0 due to the attention_mask.

This effectively cancels them out, preventing the model from paying attention to these meaningless padding elements.

In simpler terms:

The attention_mask acts like a spotlight, highlighting the important parts of the input sequence (the actual words) and dimming the unimportant parts (the padding).

It guides BERT's attention to focus only on the relevant tokens for understanding the meaning of the text.

In your specific example:

You've provided a portion of the attention_mask tensor, and it contains only 1s.

This indicates that, up to that point in the sequence, all the tokens are real words or subwords and should be considered by the attention mechanism.

If there were any padding tokens in the corresponding input_ids tensor, you would see 0s in the attention_mask at the same positions.

----------------------------------
#In simpler terms:

Imagine you have a box that can hold 512 items (the required sequence length for BERT).

You have a sentence with 10 words. You place those 10 words in the box, and to fill the remaining space (502 slots), you add empty placeholders (padding tokens represented by 0).

The '101' and '102' are special markers to indicate the beginning and end of your actual sentence within the box.

Here we have specified a few arguments that require some explanation.

* `max_length` - this tell the tokenizer the maximum number of tokens we want to see in each sample, for BERT we almost always use `512` as that is the length of sequences that BERT consumes.

* `truncation` - if our input string `txt` contains more tokens than allowed (specified in `max_length` parameter) then we cut all tokens past the `max_length` limit.

* `padding` - if our input string `txt` contains less tokens than specified by `max_length` then we pad the sequence with zeros (`0` is the token ID for *'[PAD]'* - BERTs padding token).

* `add_special_tokens` - whether or not to add special tokens, when using BERT we always want this to be `True` unless we are adding them ourselves.

| Token | ID | Description |
| --- | --- | --- |
| [PAD] | 0 | Used to fill empty space when input sequence is shorter than required sequence size for model |
| [UNK] | 100 | If a word/character is not found in BERTs vocabulary it will be represented by this *unknown* token |
| [CLS] | 101 | Represents the start of a sequence |
| [SEP] | 102 | Seperator token to denote the end of a sequence and as a seperator where there are multiple sequences |
| [MASK] | 103 | Token used for masking other tokens, used for masked language modeling |

*Note that our tokenized sequence begins with `101`, the seperator token `102` can be found seperating the input sequence and padding tokens `0`.*

* `return_tensors` - here we specify either `'pt'` to return PyTorch tensors, or `'tf'` to return TensorFlow tensors.

The output produced includes **three** tensors in a dictionary format, `'input_ids'`, `'token_type_ids'`, and `'attention_mask'`. We can ignore `'token_type_ids'` as they are not used by BERT, the other two tensors are however.

* `'input_ids'` are the token ID representations of our input text. These will be passed into an embedding array where vector representations of each word will be found and passed into the following BERT layers.

* `'attention_mask'` tells the attention layers in BERT which words to calculate attention for. If you look at this tensors you will see that each `1` value maps to an input ID from the `'input_ids'` tensor, whereas each `0` value maps to a *padding token* from the `'attention_mask'` tensor. In the attention layer (activations mapping to padding tokens are multiplied by 0, and so are cancelled out).

Now that we have our tokenized input, we can pass it onto our `model` for inference. We pass in our `tokens` as *\*\*kwargs* (key word arguments), which we can use thanks to our tokens being in a dictionary format. When a dictionary is passed as a \*\*kwargs argument, the keys will be taken literally as variable names and the respective values become the variable values. So these two approaches would do the same thing:

```
# without **kwargs
random_func(var1='hello', var2='world')

# with **kwargs
input_dict = {'var1': 'hello', 'var2': 'world'}
random_func(**input_dict)
```

Let's see how that looks for making predictions with our model.

# Understanding the use of `kwargs`**

##**kwargs (keyword arguments)

 is a way to pass a variable number of keyword arguments to a function.

It's essentially a dictionary where the keys become the argument names, and the values become the argument values.

-------------------------
How it Works in Your Example

input_dict: You create a dictionary called input_dict with two key-value pairs: 'var1': 'hello' and 'var2': 'world'.

`random_func(input_dict):** When you callrandom_funcwith**input_dict`, Python unpacks the dictionary.

The key 'var1' is treated as an argument name, and its value 'hello' is assigned to that argument.

Similarly, the key 'var2' becomes an argument name, and 'world' is assigned to it.

Equivalent Call

The code you provided is functionally equivalent to this:


random_func(var1='hello', var2='world')

----------------------

#Benefits of Using `kwargs`**

##Flexibility:
You can easily pass different sets of arguments to the function without changing its definition.

##Readability:
It can make function calls more clear when you have many arguments.

##Extensibility:
You can add new arguments to the dictionary without modifying the function call.

---------------------------
#Important Considerations

#Function Definition:

The function you're calling (random_func in this case) needs to be designed to accept keyword arguments.

#Argument Names:
The keys in the dictionary must match the expected argument names in the function definition.

In [ ]:
output = model(**tokens)

output

SequenceClassifierOutput(loss=None, logits=tensor([[-1.8200,  2.4484,  0.0216]], grad_fn=<AddmmBackward>), hidden_states=None, attentions=None)

You will notice here that the output logits tensor **cannot** be a set of probabilities because probability values must be within the range 0-1. These are infact the final output activations from BERT, to convert these into probabilities we must apply a **softmax** function. We will be using the PyTorch implementation of softmax for this, which we import from `torch.nn.functional`.

In [ ]:
import torch.nn.functional as F

# apply softmax to the logits output tensor of our model (in index 0) across dimension -1
probs = F.softmax(output[0], dim=-1)

probs

tensor([[0.0127, 0.9072, 0.0801]], grad_fn=<SoftmaxBackward>)

*(We use `dim=-1` as **-1** signifies our tensors final dimension, so if we had a 3D tensor with dims `[0, 1, 2]` writing `dim=-1` is the equivalent to writing `dim=2`. In this case if we wrote `dim=-2` this would be the equivalent to writing `dim=1`. For a 2D tensor with dims `[0, 1]`, `dim=-1` is the equivalent of `dim=1`.)*

Now we have a tensor containing three classes, all with outputs within the probability range of 0-1, these are our probabilities! We can see that class index **1** has the highest probability with a value of **0.9072**. We can use PyTorch's argmax function to extract this, we can use `argmax` after importing `torch`.

In [ ]:
import torch

pred = torch.argmax(probs)

pred

tensor(1)

Argmax outputs our winning class as **1** as expected. To convert this value from a PyTorch tensor to a Python integer we can use the `item` method.

In [ ]:
pred.item()

1